Setup i Konfiguracja

In [30]:
# --- 1. SETUP I IMPORTY ---
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torchvision import datasets, transforms, models
from torch.utils.data import Subset, DataLoader
from tqdm.auto import tqdm
from google.colab import drive, userdata

# Konfiguracja stałych
BASE_PATH = '/content/drive/MyDrive/Projekty_Studia'
REPO_NAME = 'dobrePraktykiProgramowania'
PROJECT_PATH = os.path.join(BASE_PATH, REPO_NAME)
RESULTS_FILE = 'raport_cnn_final.csv'

# Montowanie dysku
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Tworzenie folderów roboczych
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)

print(f"✅ Środowisko gotowe. Ścieżka projektu: {PROJECT_PATH}")

✅ Środowisko gotowe. Ścieżka projektu: /content/drive/MyDrive/Projekty_Studia/dobrePraktykiProgramowania


Logic Core

In [31]:
# --- 2. KLASA ZARZĄDZAJĄCA EKSPERYMENTAMI ---
class ExperimentRunner:
    def __init__(self, device_type='cuda'):
        self.device = torch.device('cuda' if torch.cuda.is_available() and device_type == 'cuda' else 'cpu')
        print(f"⚙️  Inicjalizacja na: {self.device}")

    def get_data_loaders(self, batch_size=64, use_augmentation=False,
                         use_normalization=True, image_size=96, data_fraction=1.0):
        # Transformacje bazowe (Resize + Tensor)
        transforms_list = [transforms.Resize((image_size, image_size)), transforms.ToTensor()]

        # [Wymaganie 4a] Normalizacja
        if use_normalization:
            transforms_list.append(transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))

        # [Wymaganie 4b] Augmentacja
        if use_augmentation:
            train_transforms = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.2),
            ] + transforms_list)
        else:
            train_transforms = transforms.Compose(transforms_list)

        test_transforms = transforms.Compose(transforms_list)

        # Pobieranie danych (do folderu ./data, który powinien być w .gitignore)
        trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
        testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transforms)

        # [Wymaganie 4d] Mniejsza partia danych
        if data_fraction < 1.0:
            indices = list(range(int(len(trainset) * data_fraction)))
            trainset = Subset(trainset, indices)

        # num_workers=0 dla stabilności w Colab
        return (DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0),
                DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0))

    def build_model(self, model_name='resnet50', num_classes=10, dropout_rate=0.0, use_transfer_learning=False):
        # [Wymaganie 4g] Różne struktury
        weights = None
        model = None

        # Wybór wag dla Transfer Learningu [Wymaganie 3b]
        if model_name == 'resnet50':
            weights = models.ResNet50_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.resnet50(weights=weights)
            in_features = model.fc.in_features
            if use_transfer_learning:
                 for param in model.parameters(): param.requires_grad = False
            # [Wymaganie 4c] Dropout
            model.fc = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))

        elif model_name == 'vgg16':
            weights = models.VGG16_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.vgg16(weights=weights)
            in_features = model.classifier[6].in_features
            if use_transfer_learning:
                for param in model.features.parameters(): param.requires_grad = False
            model.classifier[6] = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))

        elif model_name == 'mobilenet':
            weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.mobilenet_v3_large(weights=weights)
            in_features = model.classifier[3].in_features
            if use_transfer_learning:
                for param in model.features.parameters(): param.requires_grad = False
            model.classifier[3] = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))

        elif model_name == 'densenet':
            weights = models.DenseNet121_Weights.IMAGENET1K_V1 if use_transfer_learning else None
            model = models.densenet121(weights=weights)
            in_features = model.classifier.in_features
            if use_transfer_learning:
                for param in model.features.parameters(): param.requires_grad = False
            model.classifier = nn.Sequential(nn.Dropout(p=dropout_rate), nn.Linear(in_features, num_classes))

        return model.to(self.device)

    def run_training(self, model, trainloader, testloader, epochs=1, limit_batches=None, target_accuracy=None):
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

        start_time = time.time()
        model.train()
        history = []

        print(f"   ▶ Start (Epochs: {epochs}, Cel: {target_accuracy if target_accuracy else '-'}%)")

        for epoch in range(epochs):
            batches_done = 0
            # Pasek postępu
            pbar = tqdm(trainloader, desc=f"Epoka {epoch+1}/{epochs}", leave=False)

            for inputs, labels in pbar:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                batches_done += 1

                # [Logika Baseline CPU] Przerwanie po X batchach dla estymacji
                if limit_batches and batches_done >= limit_batches:
                    elapsed = time.time() - start_time
                    estimated_total = elapsed * (len(trainloader) / limit_batches)
                    return estimated_total, 0.0 # Zwracamy 0 accuracy bo nie testujemy

            # Ewaluacja po epoce
            if testloader:
                correct, total = 0, 0
                model.eval()
                with torch.no_grad():
                    for data in testloader:
                        images, labels = data[0].to(self.device), data[1].to(self.device)
                        outputs = model(images)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                acc = 100 * correct / total
                history.append(acc)
                model.train()
                print(f"      Dokładność: {acc:.2f}%")

                # [Wymaganie 3b] Przerwanie po osiągnięciu progu
                if target_accuracy and acc >= target_accuracy:
                    return time.time() - start_time, acc

        total_time = time.time() - start_time
        final_acc = history[-1] if history else 0.0
        return total_time, final_acc

Uruchomienie Eksperymentów

In [32]:
# --- 3. WYKONANIE TESTÓW ---
all_results = []

def log_result(category, time_s, acc, details):
    """Funkcja pomocnicza do logowania wyników"""
    entry = {
        'Kategoria': category,
        'Czas (s)': round(time_s, 2),
        'Dokładność (%)': round(acc, 2),
        'Szczegóły': details
    }
    all_results.append(entry)
    print(f"📊 ZAPISANO: {category} | {acc:.2f}% | {time_s:.2f}s | {details}")


runner = ExperimentRunner(device_type='cuda')

# A. BASELINE (CPU vs GPU)
print("\n--- 1. BASELINE ---")
# CPU (Szybka estymacja na 20 batchach)
runner_cpu = ExperimentRunner('cpu')
l_cpu, _ = runner_cpu.get_data_loaders(batch_size=32)
m_cpu = runner_cpu.build_model('resnet50')
t_cpu, _ = runner_cpu.run_training(m_cpu, l_cpu, None, epochs=1, limit_batches=20)
log_result('Baseline', t_cpu, 0.0, 'CPU (Estymacja czasu)')

# GPU (Pełna epoka)
l_gpu, tl_gpu = runner.get_data_loaders(batch_size=64)
m_gpu = runner.build_model('resnet50')
t_gpu, acc_gpu = runner.run_training(m_gpu, l_gpu, tl_gpu, epochs=1)
log_result('Baseline', t_gpu, acc_gpu, 'GPU (ResNet50)')

# B. STRUKTURY SIECI
print("\n--- 2. STRUKTURY SIECI ---")
for net in ['resnet50', 'vgg16', 'mobilenet', 'densenet']:
    # Mniejszy rozmiar obrazu dla szybkości testu (64x64)
    l, tl = runner.get_data_loaders(batch_size=64, image_size=64)
    m = runner.build_model(net)
    t, acc = runner.run_training(m, l, tl, epochs=1)
    log_result('Struktura', t, acc, net)

# C. TRANSFER LEARNING
print("\n--- 3. TRANSFER LEARNING ---")
# Cel: 80% accuracy
m_tl = runner.build_model('resnet50', use_transfer_learning=True)
t_tl, acc_tl = runner.run_training(m_tl, l_gpu, tl_gpu, epochs=10, target_accuracy=80.0)
log_result('Transfer Learning', t_tl, acc_tl, 'Pretrained (Frozen Weights)')

# D. OPTYMALIZACJE (Przykłady: Normalizacja, Augmentacja)
print("\n--- 4. OPTYMALIZACJE ---")
# Bez normalizacji
l_no_norm, tl_no_norm = runner.get_data_loaders(use_normalization=False, image_size=64)
m = runner.build_model('resnet50')
t, acc = runner.run_training(m, l_no_norm, tl_no_norm, epochs=1)
log_result('Normalizacja', t, acc, 'BEZ Normalizacji')

# --- ZAPIS DO PLIKU ---
df = pd.DataFrame(all_results)
df.to_csv(RESULTS_FILE, index=False)
print(f"\n✅ Raport zapisany jako {RESULTS_FILE}")
print(df)

⚙️  Inicjalizacja na: cuda

--- 1. BASELINE ---
⚙️  Inicjalizacja na: cpu
   ▶ Start (Epochs: 1, Cel: -%)


Epoka 1/1:   0%|          | 0/1563 [00:00<?, ?it/s]

📊 ZAPISANO: Baseline | 0.00% | 3972.83s | CPU (Estymacja czasu)
   ▶ Start (Epochs: 1, Cel: -%)


Epoka 1/1:   0%|          | 0/782 [00:00<?, ?it/s]

      Dokładność: 36.15%
📊 ZAPISANO: Baseline | 36.15% | 53.37s | GPU (ResNet50)

--- 2. STRUKTURY SIECI ---
   ▶ Start (Epochs: 1, Cel: -%)


...

      Dokładność: 79.64%


Epoka 10/10:   0%|          | 0/782 [00:00<?, ?it/s]

      Dokładność: 79.90%
📊 ZAPISANO: Transfer Learning | 79.90% | 414.58s | Pretrained (Frozen Weights)

--- 4. OPTYMALIZACJE ---
   ▶ Start (Epochs: 1, Cel: -%)


Epoka 1/1:   0%|          | 0/782 [00:00<?, ?it/s]

      Dokładność: 35.11%
📊 ZAPISANO: Normalizacja | 35.11% | 34.41s | BEZ Normalizacji

✅ Raport zapisany jako raport_cnn_final.csv
           Kategoria  Czas (s)  Dokładność (%)                    Szczegóły
0           Baseline   3972.83            0.00        CPU (Estymacja czasu)
1           Baseline     53.37           36.15               GPU (ResNet50)
2          Struktura     42.12           36.95                     resnet50
3          Struktura     53.59           36.77                        vgg16
4          Struktura     43.58           31.00                    mobilenet
5          Struktura     72.21           47.98                     densenet
6  Transfer Learning    414.58           79.90  Pretrained (Frozen Weights)
7       Normalizacja     34.41           35.11             BEZ Normalizacji


Kod do obsługi Git

In [38]:
# --- 4. INTEGRACJA Z GITHUBEM (MLOPS) ---
from google.colab import userdata
import shutil

# --- KONFIGURACJA UŻYTKOWNIKA (Zmień na swoje dane!) ---
GIT_USER = "Rafalbanas"
GIT_EMAIL = "rafabanas40@gmail.com"
REPO_URL = "github.com/Rafalbanas/dobrePraktykiProgramowania.git"
COMMIT_MESSAGE = "feat: update CNN optimization results and notebook"  # Zgodne z conventional commits

def push_to_github():
    print("🚀 Rozpoczynam synchronizację z GitHub...")

    # 1. Pobranie tokenu z sekretów Colab
    try:
        git_token = userdata.get('GITHUB_TOKEN')
    except Exception:
        print("❌ BŁĄD: Nie znaleziono tokenu 'GITHUB_TOKEN' w sekretach Colab!")
        return

    # 2. Konfiguracja ścieżek
    repo_name = REPO_URL.split('/')[-1].replace('.git', '')
    repo_path = os.path.join(BASE_PATH, repo_name)

    # Skonstruowanie bezpiecznego URL z tokenem
    auth_url = f"https://{git_token}@{REPO_URL}"

    # 3. Klonowanie lub Pull
    if not os.path.exists(repo_path):
        print(f"📥 Klonowanie repozytorium do {repo_path}...")
        !git clone {auth_url} "{repo_path}"
    else:
        print("🔄 Aktualizacja repozytorium (git pull)...")
        %cd "{repo_path}"
        !git pull

    # 4. Konfiguracja tożsamości Gita
    %cd "{repo_path}"
    !git config user.email "{GIT_EMAIL}"
    !git config user.name "{GIT_USER}"

    # 5. Tworzenie .gitignore (Ważne! Ignorujemy duże pliki danych)
    gitignore_path = os.path.join(repo_path, ".gitignore")
    with open(gitignore_path, "w") as f:
        f.write("data/\n*.pth\n__pycache__/\n.ipynb_checkpoints/\n")

    # 6. Kopiowanie plików do repozytorium
    # Kopiujemy plik z wynikami CSV
    if os.path.exists(RESULTS_FILE):
        try:
            shutil.copy(RESULTS_FILE, repo_path)
            print("✅ Skopiowano raport CSV.")
        except shutil.SameFileError:
            print("ℹ️ Raport CSV już znajduje się w folderze docelowym.")

    # Kopiujemy bieżący notatnik (Zakładamy, że jest na Drive w BASE_PATH)
    # UWAGA: Zapisz notatnik (Ctrl+S) przed uruchomieniem tej komórki!
    notebook_name = "Banas_testowanie i optymalizacja sieci CNN.ipynb" # Sprawdź czy nazwa się zgadza
    source_notebook = os.path.join(BASE_PATH, notebook_name)

    if os.path.exists(source_notebook):
        try:
            shutil.copy(source_notebook, repo_path)
            print("✅ Skopiowano notatnik.")
        except shutil.SameFileError:
             print("ℹ️ Notatnik już znajduje się w folderze docelowym.")
    else:
        # Próba znalezienia notatnika w bieżącym katalogu Colab
        try:
            current_nb = [f for f in os.listdir('/content/drive/MyDrive/Colab Notebooks/') if 'CNN' in f][0] # Przybliżone szukanie
            shutil.copy(os.path.join('/content/drive/MyDrive/Colab Notebooks/', current_nb), os.path.join(repo_path, notebook_name))
            print(f"✅ Znaleziono i skopiowano: {current_nb}")
        except:
            print("⚠️ Nie znaleziono pliku .ipynb do skopiowania.")

    # 7. Git Add, Commit, Push
    !git add .
    !git commit -m "{COMMIT_MESSAGE}"
    !git push origin main
    print("🚀 Zmiany wysłane na GitHub!")

# --- TU DODAJ WYWOŁANIE FUNKCJI ---
push_to_github()

🚀 Rozpoczynam synchronizację z GitHub...
🔄 Aktualizacja repozytorium (git pull)...
/content/drive/MyDrive/Projekty_Studia/dobrePraktykiProgramowania
Already up to date.
/content/drive/MyDrive/Projekty_Studia/dobrePraktykiProgramowania


SameFileError: 'raport_cnn_final.csv' and '/content/drive/MyDrive/Projekty_Studia/dobrePraktykiProgramowania/raport_cnn_final.csv' are the same file